In [1]:
!pip install xgboost

In [1]:
import pandas as pd
import urllib.request


import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, make_scorer

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
RAND=1488

In [2]:
import warnings
warnings.filterwarnings('ignore')

# FIRST DATASET

In [14]:
from ucimlrepo import fetch_ucirepo 

wine = fetch_ucirepo(id=109) 
X = wine.data.features 
y = wine.data.targets 

In [65]:
X

,Alcohol,Malicacid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,0D280_0D315_of_diluted_wines,Proline
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


In [66]:
indices_to_drop = y[y == 3].dropna().index

X = X.drop(indices_to_drop)
y = y.drop(indices_to_drop)
y['class'] = y['class'] == 2

In [67]:
for col in X.columns:
    median_val = X[col].median()
    X.loc[:, col] = X[col] >= median_val

In [13]:
Models = ["DecisionTreeClassifier", "RandomForestClassifier", "XGBClassifier", "KNeighborsClassifier", "LogisticRegression"]
data = []

In [69]:
data_NO = 1

In [22]:
def train(X_df, y, model0, params, spliter, scores=None):
    if scores == None:
        scores=["accuracy", "f1"]
    clf = GridSearchCV(model0, params, scoring=scores, refit="f1", cv=spliter)
    clf.fit(X,y)
    best_model = clf.best_estimator_
    ind = clf.best_index_
    res = [clf.best_params_]
    for sc in scores:
        res.append(clf.cv_results_["mean_test_"+sc][ind])
    return res

spliter = KFold(n_splits=5, shuffle=True, random_state=RAND)

In [71]:
model1 = DecisionTreeClassifier(random_state=RAND)
params = \
    {"max_depth": [5, 6, 7, 8, 9, 10],
     "min_samples_split": [2, 4, 5, 6]}

res=[Models[0], data_NO]
res.extend(train(X, y, model1, params, spliter, scores={"acc" : make_scorer(accuracy_score), "f1" : make_scorer(f1_score, average="macro")}))
data.append(res)
data[-1]

['DecisionTreeClassifier',
 1,
 {'max_depth': 5, 'min_samples_split': 6},
 0.9615384615384617,
 0.9608446833930705]

In [72]:
model2 = RandomForestClassifier(random_state=RAND)
params = \
    {"n_estimators": [25, 30, 35, 40],
     "max_depth": [3, 4, 5],
     "min_samples_split" : [2, 3, 4, 5]}
res=[Models[1], data_NO]
res.extend(train(X, y, model2, params, spliter, scores={"acc" : make_scorer(accuracy_score), "f1" : make_scorer(f1_score, average="macro")}))
data.append(res)
data[-1]

['RandomForestClassifier',
 1,
 {'max_depth': 3, 'min_samples_split': 5, 'n_estimators': 25},
 0.9692307692307693,
 0.9688323946065882]

In [73]:
model3 = XGBClassifier(random_state=RAND)
params = \
    {"n_estimators": [3, 5, 6, 7, 9, 11],
     "max_depth": [2, 3, 4, 5],
     "learning_rate" : [0.5, 0.7, 0.9, 1.1]}
res=[Models[2], data_NO]
res.extend(train(X, y, model3, params, spliter, scores={"acc" : make_scorer(accuracy_score), "f1" : make_scorer(f1_score, average="macro")}))
data.append(res)
data[-1]

['XGBClassifier',
 1,
 {'learning_rate': 0.7, 'max_depth': 2, 'n_estimators': 9},
 0.976923076923077,
 0.976512483133753]

In [74]:
model4 = KNeighborsClassifier()
params = \
    {"n_neighbors": [3, 4, 5, 7, 10]}
res=[Models[3], data_NO]
res.extend(train(X, y, model4, params, spliter, scores={"acc" : make_scorer(accuracy_score), "f1" : make_scorer(f1_score, average="macro")}))
data.append(res)
data[-1]

['KNeighborsClassifier',
 1,
 {'n_neighbors': 3},
 0.9615384615384615,
 0.9607522851477487]

In [75]:
model5 = LogisticRegression()
params = \
    {'penalty': ['l2'],
    'max_iter': [50, 75, 100, 150, 200, 350]}
res=[Models[4], data_NO]
res.extend(train(X, y, model5, params, spliter, scores={"acc" : make_scorer(accuracy_score), "f1" : make_scorer(f1_score, average="macro")}))
data.append(res)
data[-1]

['LogisticRegression',
 1,
 {'max_iter': 50, 'penalty': 'l2'},
 0.9615384615384617,
 0.9607522851477487]

# SECOND DATASET

In [76]:
wine_quality = fetch_ucirepo(id=186) 

X = wine_quality.data.features 
y = wine_quality.data.targets 

In [77]:
X

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8


In [79]:
for col in X.columns:
    median_val = X[col].median()
    X.loc[:, col] = X[col] >= median_val

median_val = y['quality'].median()
y.loc[:, 'quality'] = y['quality'] >= median_val

In [80]:
data_NO = 2

In [81]:
model1 = DecisionTreeClassifier(random_state=RAND)
params = \
    {"max_depth": [5, 6, 7, 8, 9, 10, 15, 20, 30, 35, 40],
     "min_samples_split": [1, 2, 4, 5, 10]}

res=[Models[0], data_NO]
res.extend(train(X, y, model1, params, spliter, scores={"acc" : make_scorer(accuracy_score), "f1" : make_scorer(f1_score, average="weighted")}))
data.append(res)
data[-1]

['DecisionTreeClassifier',
 2,
 {'max_depth': 7, 'min_samples_split': 2},
 0.731875762420797,
 0.7305845498190744]

In [82]:
model2 = RandomForestClassifier(random_state=RAND)
params = \
    {"n_estimators": [50, 100, 150],
     "max_depth": [5, 6, 7],
     "min_samples_split" : [2, 3]}
res=[Models[1], data_NO]
res.extend(train(X, y, model2, params, spliter, scores={"acc" : make_scorer(accuracy_score), "f1" : make_scorer(f1_score, average="weighted")}))
data.append(res)
data[-1]

['RandomForestClassifier',
 2,
 {'max_depth': 7, 'min_samples_split': 3, 'n_estimators': 150},
 0.7434202641084859,
 0.7364235877686385]

In [84]:
model3 = XGBClassifier(random_state=RAND)
params = \
    {"n_estimators": [3, 5, 6, 7, 9, 11],
     "max_depth": [2, 3, 4, 5],
     "learning_rate" : [0.5, 0.7, 0.9, 1.1]}
res=[Models[2], data_NO]
res.extend(train(X, y, model3, params, spliter, scores={"acc" : make_scorer(accuracy_score), "f1" : make_scorer(f1_score, average="weighted")}))
data.append(res)
data[-1]

['XGBClassifier',
 2,
 {'learning_rate': 0.5, 'max_depth': 5, 'n_estimators': 9},
 0.7452691419435069,
 0.7417482800212071]

In [85]:
model4 = KNeighborsClassifier()
params = \
    {"n_neighbors": [3, 4, 5, 7, 10]}
res=[Models[3], data_NO]
res.extend(train(X, y, model4, params, spliter, scores={"acc" : make_scorer(accuracy_score), "f1" : make_scorer(f1_score, average="weighted")}))
data.append(res)
data[-1]

['KNeighborsClassifier',
 2,
 {'n_neighbors': 10},
 0.7257209687925623,
 0.7257117441528619]

In [86]:
model5 = LogisticRegression()
params = \
    {'penalty': ['l2'],
    'max_iter': [50, 75, 100]}
res=[Models[4], data_NO]
res.extend(train(X, y, model5, params, spliter, scores={"acc" : make_scorer(accuracy_score), "f1" : make_scorer(f1_score, average="weighted")}))
data.append(res)
data[-1]

['LogisticRegression',
 2,
 {'max_iter': 50, 'penalty': 'l2'},
 0.7212577722508439,
 0.7137339950722925]

# THIRD DATASET

In [15]:
adult = fetch_ucirepo(id=2)
X = adult.data.features 
y = adult.data.targets 

In [16]:
X

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States
48838,64,NaN,321403,HS-grad,9,Widowed,NaN,Other-relative,Black,Male,0,0,40,United-States
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States


In [17]:
X['workclass'] = X['workclass'].fillna('?')
X['occupation'] = X['occupation'].fillna('?')
X['native-country'] = X['native-country'].fillna('?')

In [18]:
y['income'] = y['income'] == '<=50K'

def convert_to_bool(column):
    if column.dtype == 'object':
        mode = column.mode()[0]
        return column == mode
    else:
        median = column.median()
        return column >= median

X = X.iloc[:, 1:].apply(convert_to_bool)

In [19]:
X

,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,False,False,False,True,False,False,False,True,True,True,True,True,True
1,False,False,False,True,True,False,True,True,True,True,True,False,True
2,True,True,True,False,False,False,False,True,True,True,True,True,True
3,True,True,False,False,True,False,True,False,True,True,True,True,True
4,True,True,False,True,True,True,False,False,False,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,True,True,False,True,False,True,False,True,False,True,True,False,True
48838,False,True,True,False,False,False,False,False,True,True,True,True,True
48839,True,True,False,True,True,True,True,True,True,True,True,True,True
48840,True,False,False,True,False,False,False,False,True,True,True,True,True


In [20]:
data_NO = 3

In [27]:
model1 = DecisionTreeClassifier(random_state=RAND)
params = \
    {"max_depth": [5, 10, 15],
     "min_samples_split": [2, 4, 5]}

res=[Models[0], data_NO]
res.extend(train(X, y, model1, params, spliter, scores={"acc" : make_scorer(accuracy_score), "f1" : make_scorer(f1_score)}))
data.append(res)
data[-1]

['DecisionTreeClassifier',
 3,
 {'max_depth': 5, 'min_samples_split': 2},
 0.6366446423437926,
 0.6704662419159959]

In [28]:
model2 = RandomForestClassifier(random_state=RAND)
params = \
    {"n_estimators": [50, 100, 150],
     "max_depth": [5, 6, 7],
     "min_samples_split" : [2, 3]}
res=[Models[1], data_NO]
res.extend(train(X, y, model2, params, spliter, scores={"acc" : make_scorer(accuracy_score), "f1" : make_scorer(f1_score)}))
data.append(res)
data[-1]

['RandomForestClassifier',
 3,
 {'max_depth': 7, 'min_samples_split': 2, 'n_estimators': 100},
 0.6348019072683828,
 0.6719955283110995]

In [29]:
model3 = XGBClassifier(random_state=RAND)
params = \
    {"n_estimators": [3, 5, 6, 7, 9, 11],
     "max_depth": [2, 3, 4, 5],
     "learning_rate" : [0.5, 0.7, 0.9, 1.1]}
res=[Models[2], data_NO]
res.extend(train(X, y, model3, params, spliter, scores={"acc" : make_scorer(accuracy_score), "f1" : make_scorer(f1_score)}))
data.append(res)
data[-1]

['XGBClassifier',
 3,
 {'learning_rate': 0.5, 'max_depth': 3, 'n_estimators': 3},
 0.6359894563600164,
 0.6750761480430384]

In [30]:
model4 = KNeighborsClassifier()
params = \
    {"n_neighbors": [3, 4, 5, 7, 10]}
res=[Models[3], data_NO]
res.extend(train(X, y, model4, params, spliter, scores={"acc" : make_scorer(accuracy_score), "f1" : make_scorer(f1_score)}))
data.append(res)
data[-1]

['KNeighborsClassifier',
 3,
 {'n_neighbors': 7},
 0.6034354606982307,
 0.6884061317131753]

In [31]:
model5 = LogisticRegression()
params = \
    {'penalty': ['l2'],
    'max_iter': [50, 75, 100]}
res=[Models[4], data_NO]
res.extend(train(X, y, model5, params, spliter, scores={"acc" : make_scorer(accuracy_score), "f1" : make_scorer(f1_score)}))
data.append(res)
data[-1]

['LogisticRegression',
 3,
 {'max_iter': 50, 'penalty': 'l2'},
 0.6248718933154391,
 0.6326510569761636]

In [ ]:
report